In [ ]:
# Extraherar från data filen som kan hämtas online 

import tarfile
import os

archive_path = "./data/cifar-10-python.tar.gz"
extract_path = "./data"

# Extract
with tarfile.open(archive_path, "r:gz") as tar:
    tar.extractall(path=extract_path)


print("Files in data folder after extraction:", os.listdir(extract_path))

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
subset_size = 5000
batch_size = 50

# Normalisera (0, 255) -> (-1, 1)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Hämta data från ./data
trainset_full = torchvision.datasets.CIFAR10(root="./data", train=True, download=False, transform=transform)
testset_full = torchvision.datasets.CIFAR10(root="./data", train=False, download=False, transform=transform)

# tar första subset_size st samples för att använda
train_subset = torch.utils.data.Subset(trainset_full, range(subset_size))
test_subset = torch.utils.data.Subset(testset_full, range(subset_size))  # same number for simplicity

# batch uppdeling för cnn
cnn_train = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
cnn_test = torch.utils.data.DataLoader(test_subset, batch_size=batch_size, shuffle=True)

# SVM behöver feature vector, tensor 32x32x3 -> 3072 features
def dataset_to_numpy(dataset):
    X = np.array([np.array(img).flatten() for img, _ in dataset]) # data
    y = np.array([label for _, label in dataset]) #labels
    return X, y

X_train, y_train = dataset_to_numpy(train_subset)
X_test, y_test = dataset_to_numpy(test_subset)

# normalisera data för svm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) # ingen data leak i testset

print("Training samples:", X_train.shape[0])
print("Input dimensions (features):", X_train.shape[1])
print("Max svm pixel range after scaling:", (X_train.min(), X_train.max()))

Training samples: 5000
Input dimensions (features): 3072
Max svm pixel range after scaling: (-2.2089858, 2.6816778)


In [3]:

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1) # padding ska behålla 32x32
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10) # 3072 -> 10 klasser

    def forward(self, x):
        x = F.relu(self.conv1(x)) # (N, 16, 32, 32)
        x = self.pool(F.relu(self.conv2(x))) # 32x32 -> 16x16  (N, 32, 16, 16)
        x = self.pool(x) # 16x16 -> 8x8  (N, 32, 8, 8)
        x = x.view(batch_size, 32 * 8 * 8)  # (N, 32, 8, 8) -> (N, 2048) Härifrån vanligt NN
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss() # logsoftmax + NLLLoss
optimizer = optim.Adam(net.parameters(), lr=0.001) # adam borde vara mycket snabbare än sgd

# 1 epok
print("Training CNN")
for epoch in range(1):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(cnn_train, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("Running loss for epoch", epoch+1, ":", running_loss)

print("Finished training")

# Räkna accuracy på test_set
correct, total = 0, 0
net.eval()
with torch.no_grad():
    for images, labels in cnn_test:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

cnn_acc = 100 * correct / total
print("CNN test accuracy on" , subset_size, "samples: ", cnn_acc, "%")


Training CNN
Running loss for epoch 1 : 194.05530071258545
Finished training
CNN test accuracy on 5000 samples:  40.5 %


In [5]:


# RBF kernel
svm = SVC(kernel='rbf', C=1.0, gamma='scale')


# Träna och predict
print("Training SVM")
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred) * 100

print("SVM test accuracy on" , subset_size, "samples: ", svm_acc, "%")


Training SVM
SVM test accuracy on 5000 samples:  45.58 %
